In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from helpers import MetadataProcesor, MakePlots
import pandas as pd

In [ ]:
N_TRIES = 1
K_VALUES = range(10, 101, 5)

In [ ]:
LFM_metadata_file_name = 'metadata_LFM'
DEEZER_metadata_file_name = 'metadata_DEEZER_musicbrainz'

metadata = MetadataProcesor(LFM_metadata_file_name, DEEZER_metadata_file_name)
metadata.process()

In [ ]:
plots_maker = MakePlots(metadata.metadata, k_values=K_VALUES, matadata_filename = DEEZER_metadata_file_name)
plots_maker.load_predictions(n_tries = N_TRIES)
plots_maker.load_datasets()

plots_maker.plot_dataset_local_streams_percents(save=True)
plots_maker.plot_local_listening_distribution_hist(save=True)

plots_maker.compute_reco_results()
plots_maker.plot_bias_topk_k_reco(save=True)